# What
分类任务，支持两种模式
1. Folder模式，需要输入`train`, `valid`两个测试集对应的目录。`labels.txt`，需要训练的label，里面每个类别一行。
2. List模式，需要输入`train`, `valid`两个测试集对应的训练文件，每行一个样本。`labels.txt`是可选参数，里面每个类别一行。`data_pattern`一个通用的目录，与train、val中的第一列进行拼接。

### List模式

在Onekey中List模式一般是采用labelme标注出来的结果，如果要使用自己的数据应用List模式，需要根据自己的实际情况对数据进行处理。

* `train.txt`，训练数据列表，中间用\t（Tab水平制表符）进行分割。
* `val.txt`，验证数据列表，中间用\t（Tab水平制表符）进行分割。
* `labels.txt`，label的集合，表明训练数据多少标签。
* `data_pattern`参数，所有数据存在的目录的公共前缀，如果`train.txt`,`val.txt`文件里面存放的是绝对路径，`data_pattern`设置为None即可。

In [ ]:
import os
from collections import namedtuple

from onekey_algo.classification3d.run_classification3d import main as clf_main3d
from onekey_algo import get_param_in_cwd
# 设置参数

for cv in range(4, 5):
    save_dir =get_param_in_cwd('data_root')
    train = os.path.join(save_dir, 'split_info', f'train-RND-{cv}.txt')
    val = os.path.join(save_dir, 'split_info', f'val-RND-{cv}.txt')
    data_pattern = os.path.join(save_dir, 'Radiology', 'crop3d')
    labels_file = os.path.join(save_dir, 'split_info', 'labels.txt')
    for model_name in ['resnet18', 'resnet50', 'ShuffleNet', 'DenseNet121'][3:]:
        params = dict(train=train,
                      val=val,
                      labels_file=labels_file,
                      data_pattern=data_pattern,
                      j=6,
                      max2use=None,
                      normalize_method='imagenet',
                      model_name=model_name,
                      gpus=[0],
                      roi_size=[96, 96, 96],
                      batch_size=4,
                      epochs=48,
                      init_lr=0.001,
                      optimizer='adam',
                      retrain=None,
                      model_root=os.path.join(save_dir, 'Radiology', 'models', f"CV-{cv}"),
                      val_interval=1,
                      iters_verbose=10,
                      model_config={'groups': 1, 'blocks_args': 'efficientnet-b4'},
                      pretrained=True)
        # 训练模型
        Args = namedtuple("Args", params)
        clf_main3d(Args(**params))